<a href="https://colab.research.google.com/github/mikful/detr/blob/master/roboflow_detr_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DETR + ROBOFLOW

The following notebook implements DETR finetuning on the Roboflow COCO format dataset. 

The main implementation is taken from: https://github.com/aivclab/detr

With changes made to the coco loader to fit our dataset. See below in Dataset section for details.

# Colab Setup



In [1]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jun  9 19:12:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


# Fork DETR and setup

In [4]:
!git clone 'https://github.com/aivclab/detr'

Cloning into 'detr'...
remote: Enumerating objects: 264, done.
remote: Total 264 (delta 0), reused 0 (delta 0), pack-reused 264
Receiving objects: 100% (264/264), 14.14 MiB | 19.41 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [5]:
%cd /content/detr 

/content/detr


In [6]:
!pip install -r  requirements.txt

     |████████████████████████████████| 61kB 4.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-install-l6x9x9d3/panopticapi
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-install-l6x9x9d3/panopticapi
     |████████████████████████████████| 12.2MB 10.0MB/s 
     |████████████████████████████████| 4.5MB 49.1MB/s 
  Created wheel for submitit: filename=submitit-1.3.3-cp37-none-any.whl size=66089 sha256=8b379ed55e2475594af9aa89ae4fa4aae8bbd3144b81d4d4d7e5f2513fd6eb51
  Stored in directory: /root/.cache/pip/wheels/75/be/c1/c57fe81efce574db4e4b3d23f26c1b75bd1d6af1124f29a78d
Successfully built submitit
  Created wheel for panopticapi: filename=panopticapi-0.1-cp37-none-any.whl size=8306 sha256=7b009d06bbf56dc0bbb4a049cbf08a49fa214590b0c6a4b11a58359d7423efb4
  Stored in directory: /tmp/pip-ephem-wheel-c

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.8.1+cu101 True



# Load a model
First we have to decide if our model should be pretrained.

This greatly depends on the size of a dataset. Smaller datasets rely more on finetuning.

In [2]:
pretrained = True

if pretrained:
    # Get pretrained weights
    checkpoint = torch.hub.load_state_dict_from_url(
                url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
                map_location='cpu',
                check_hash=True)

    # Remove class weights
    del checkpoint["model"]["class_embed.weight"]
    del checkpoint["model"]["class_embed.bias"]

    # SaveOGH
    torch.save(checkpoint,
               'detr-r50_no-class-head.pth')


# Dataset
Our dataset should be loadable as a COCO format

This allows us to use the pycocotools to load the data dict for the main python script


Need to change /content/detr/datasets/coco.py build() line 147 to:


```
def build(image_set, args):
    root = Path(args.coco_path)
    assert root.exists(), f'provided COCO path {root} does not exist'
    mode = 'instances'
    PATHS = {
        "train": (root / "train", root / "train/_annotations.coco.json"),
        "val": (root / "valid", root / "valid/_annotations.coco.json"),
    }

    img_folder, ann_file = PATHS[image_set]
    dataset = CocoDetection(img_folder, ann_file, transforms=make_coco_transforms(image_set), return_masks=args.masks)
    return dataset

```



Download Roboflow dataset

In [3]:
!mkdir /content/roboflow-dataset
%cd /content/roboflow-dataset
!rm -rf train valid test README.roboflow.txt data.yaml
!curl -L "https://app.roboflow.com/ds/CcZcXC9tAY?key=19cj8EBfm3" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

mkdir: cannot create directory ‘/content/roboflow-dataset’: File exists
/content/roboflow-dataset
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   887  100   887    0     0    853      0  0:00:01  0:00:01 --:--:--   853
100 19.1M  100 19.1M    0     0  14.3M      0  0:00:01  0:00:01 --:--:-- 14.3M
Archive:  roboflow.zip
 extracting: README.roboflow.txt     
   creating: train/
 extracting: train/00f2198f-2431-485a-b15e-b8b587ce182e_jpg.rf.142724d6e8ce6ea267d0adaf15963fe3.jpg  
 extracting: train/01b9bbab-3618-4835-aff0-40e0db284981_jpg.rf.407bc18c6e841bc0a334363ced91c639.jpg  
 extracting: train/035b7ef0-df5e-4214-8fa3-f4eacbfe9a91_jpg.rf.eec5c9027fc23c5918d8ec2f224f3a72.jpg  
 extracting: train/04d6b405-3755-4063-bad2-72c0daf54334_jpeg.rf.41975e5439d14d79b6f29779de45c0ca.jpg  
 extracting: train/0892ca74-ceef-4c20-bcdc-2a6ce2be9052_jpeg.rf.96ceab9c81c78b8c68e2f73d43ed45f7

In [3]:
dataset_file = "coco" # alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py

dataDir='/content/roboflow-dataset' # should lead to a directory with a train2017 and val2017 folder as well as an annotations folder
num_classes = 3 # this int should be the actual number of classes + 1 (for no class)

outDir = 'outputs'
resume = "detr-r50_no-class-head.pth" if pretrained else ""


# Training
We use the main.py script to run our training


Note - changes made to class loss in /content/detr/detr.py line 126:

```
           losses['class_error'] = 100 - accuracy(src_logits[idx][..., :-1], target_classes_o)[0]
```




In [ ]:
%cd /content/detr

!python main.py \
  --dataset_file $dataset_file \
  --coco_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs 50 \
  --batch_size 16

/content/detr
Not using distributed mode
git:
  sha: 8830cacdc981924169546a0e59d94b6c94fd775d, status: has uncommited changes, branch: master

Namespace(aux_loss=True, backbone='resnet50', batch_size=16, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path='/content/roboflow-dataset', dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=50, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=1e-05, lr_backbone=1e-06, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_classes=3, num_queries=100, num_workers=2, output_dir='outputs', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='detr-r50_no-class-head.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
Building a DETR model with 3 classes
number of params: 41279752
l

# Results¶
Quick and easy overview of the training results

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path(outDir)]

In [ ]:

fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:

fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)